In [7]:
#Importing necessary modules 
import random
from deap import base, creator, tools, algorithms
import numpy
import math

In [ ]:
# Empezamos por declarar el fitness de cada individuo: Si es un problema
# de maximización el valor del atributo weights es 1, si por el contrario
# el problema es de minimización el valor es -1.
# Consideramos que nuestro objetivo es reducir al mínimo el número de errores
# que contenga el circuito.


In [8]:
creator.create('Fitness', base.Fitness, weights=(-1.0,))

In [9]:
#Consideramos los individuos del problema como las puertas lógicas.

In [10]:
creator.create('Individuo', list, fitness = creator.Fitness)

In [ ]:
# A continuación debemos de crear la caja de herramienta donde vamos a
# registrar los elementos necesarios para poder aplicar el algoritmo genético.

In [11]:
caja_de_herramientas = base.Toolbox()

In [18]:
#Declaramos las puertas lógicas
def puerta_or(a,b):
    return a+b

def puerta_and(a,b):
    return a*b

def puerta_not(a):
    if a==0:
        return 1
    if a==1:
        return 0

def puerta_nand(a,b):
    if a==0 and b==0:
        return 1
    elif a==0 and b==1:
        return 1
    elif a==1 and b==0:
        return 1
    elif a==1 and b==1:
        return 0

def puerta_xor(a,b):
    if a==1 and b==1:
        return 0
    return a+b

0